In [24]:
import pandas as pd
import math
from datetime import datetime, timedelta

In [25]:
Overview = pd.read_csv('overview.csv')
Overview.columns

Index(['Capacity', 'Available', 'Waitlist', 'PercentAvailable', 'Undergrad',
       'Dept', 'CourseNum', 'StudyAbroad'],
      dtype='object')

In [26]:
data = pd.read_csv("eecs-445.csv")
data

,Unnamed: 0,Section,Instruction Mode,Class No,Enroll Stat,Open Seats,Wait List,Hour
0,3546,001 (LEC),In Person,26932,Open,120,-1,2021-11-15 06:00:00
1,3546,002 (LEC),In Person,15919,Open,4,10,2021-12-09 19:00:00
2,3547,002 (LEC),In Person,24565,Open,120,-1,2021-11-15 06:00:00
3,3547,010 (DIS),In Person,15922,Closed,0,3,2021-12-09 19:00:00
4,3548,011 (DIS),In Person,22551,Open,40,-1,2021-11-15 06:00:00
...,...,...,...,...,...,...,...,...
4123,5328,014 (DIS),In Person,26935,Closed,0,13,2021-12-09 19:00:00
4124,5329,015 (DIS),In Person,18524,Open,25,-1,2021-11-15 06:00:00
4125,5329,015 (DIS),In Person,26936,Closed,0,13,2021-12-09 19:00:00
4126,5330,016 (DIS),In Person,18504,Open,25,-1,2021-11-15 06:00:00


In [27]:
# there's a chance this would be faster if we just 
    # calculated all past rates for all classes one time
    # instead of everytime the scrapper ran
# also I'm dividing by the number of days from last time here (25)
# Separating rates thoughts
    # I'd probably add a time parameter then calculate days since the registration opened
    # And then find the rate for some period around that day (consider egde cases)
    # predicted_rate() would also need a time parameter
    # predicted_final_day would probably need a loop to working back thru predicted_rate() outputs
        # from the 25th day or something until it hits a predicted postive "Open Seats"
def past_rate(sectionNumber, pastData) :
    classData = pastData[pastData['Class No']==sectionNumber]
    # this might be faster if we only sorted this once
    classData = classData.sort_values(by=['Hour'], ascending=False).reset_index(drop=True)
    datetime.strptime('2021-12-09 19:00:00', '%Y-%m-%d %H:%M:%S')
    for i in range(0, len(classData) - 1) :
        days = 25
        if classData['Open Seats'][i] > 0 :
            days = (datetime.strptime(classData['Hour'][i], '%Y-%m-%d %H:%M:%S') - 
                                     datetime.strptime(classData['Hour'][len(classData) - 1], '%Y-%m-%d %H:%M:%S')).days
        if days == 0 : days = 25
        return (classData['Open Seats'].agg('max') - classData['Open Seats'][i]) / days
    return 0
    

In [28]:
past_rate(26935, data)

1.6

In [29]:
def current_rate(sectionNumber, currentData) :
    # maybe make recent current rate ??
    classData = currentData[currentData['Class No']==sectionNumber].sort_values(by=['Hour'], ascending=False).reset_index(drop=True)
    daysList = []
    for time in currentData['Hour']:
        daysList.append(time[0:10])
    days = pd.Series(daysList).unique()
    return (classData['Open Seats'].agg('max') - classData['Open Seats'][0]) / len(days)

In [40]:
current_rate(18523, data)

0.0

In [44]:
data[data['Class No']==18523]

,Unnamed: 0,Section,Instruction Mode,Class No,Enroll Stat,Open Seats,Wait List,Hour
4118,5326,012 (DIS),In Person,18523,Open,25,-1,2021-11-15 06:00:00


In [31]:
def predicted_rate(sectionNumber, pastData, currentData) :
  # idk if this formula actually makes sense but meh
  daysList = []
  for time in currentData['Hour']:
      daysList.append(time[0:10])
  days = pd.Series(daysList).unique()
  # potential formula to make current rate count more over time
      # this formula definity won't work for incoming freshman but we can't really predict
#     return (current_rate(sectionNumber, currentData)*15 / 25) + (past_rate(sectionNumber, pastData)*(25 - 15) / 25)
  return (current_rate(sectionNumber, currentData)*len(days) / 25) + (past_rate(sectionNumber, pastData)*(25 - len(days)) / 25)

In [39]:
predicted_rate(18523, data, data)

0.0

In [33]:
def predicted_final_day(sectionNumber, pastData, currentData) :    
    classData = currentData[currentData['Class No']==sectionNumber]
    classData = classData.sort_values(by=['Hour'], ascending=False).reset_index(drop=True)
    rate = predicted_rate(sectionNumber, pastData, currentData)
    daysList = []
    for time in currentData['Hour'] :
        daysList.append(time[0:10])
    days = pd.Series(daysList).unique()
    if rate > 0 :
        days_till_closing = classData["Open Seats"].agg('max') / rate
    else :
        days_till_closing = 0
    finalDay = datetime.strptime(days[0], '%Y-%m-%d') + timedelta(math.floor(days_till_closing))
    return finalDay.strftime('%Y-%m-%d')
#     return days_till_closing

In [38]:
predicted_final_day(18523, data, data)

'2021-11-15'

In [35]:
# # Scraper output
# os.makedirs("./output/", exist_ok=True)
#     for (course, listing) in df.groupby("Course").groups.items():
#         df.loc[
#             listing,
#             [
#                 "Section",
#                 "Instruction Mode",
#                 "Class No",
#                 "Enroll Stat",
#                 "Open Seats",
#                 "Wait List",
#                 "Hour",
#             ],
#         ].to_csv(f"./output/{slugify(course)}.csv")
def output(pastData, currentData) :
    predictedFinalDays = []
    for section in currentData['Class No'] :
        predictedFinalDays.append([section, predicted_final_day(section, pastData, currentData)])
    predictedFinalDaysDF = pd.DataFrame(predictedFinalDays)
    predictedFinalDaysDF.columns = ['Class No', 'Predicted Final Day']
    classInfoDF = pd.DataFrame(currentData[["Section", "Instruction Mode", "Class No", "Enroll Stat"]])
    return pd.merge(predictedFinalDaysDF, classInfoDF, on='Class No',how='outer')
    # Other Ideas:
    # we may wanna add an all zero "waitlist" column or something to classInfo idk
    # for intermediate values ? :
    # loop through remaining hours : 
        # loop through "Class No", calculate remaining using above fcns "Open Seats"
        # join classInfo to the open seats calculation result
        # export

In [36]:
output(data, data)

,Class No,Predicted Final Day,Section,Instruction Mode,Enroll Stat
0,26932,2021-12-11,001 (LEC),In Person,Open
1,26932,2021-12-11,001 (LEC),In Person,Open
2,26932,2021-12-11,001 (LEC),In Person,Open
3,26932,2021-12-11,001 (LEC),In Person,Open
4,26932,2021-12-11,001 (LEC),In Person,Open
...,...,...,...,...,...
2113579,18523,2021-11-15,012 (DIS),In Person,Open
2113580,18501,2021-11-15,013 (DIS),In Person,Open
2113581,18502,2021-11-15,014 (DIS),In Person,Open
2113582,18524,2021-11-15,015 (DIS),In Person,Open


In [37]:
date1 = datetime.strptime('2021-12-09 20:00:00', '%Y-%m-%d %H:%M:%S')
date2 = datetime.strptime('2021-12-09 19:00:00', '%Y-%m-%d %H:%M:%S')
(date1 - date2).days

0